In [1]:
# 사용자 행동 인식 데이터 세트에 대한 예측 분류를 수행해 보겟습니다.
# 30 명에게 스마트폰 센서를 장착한 뒤 사람의 동작과 관련된 여러 가지 피처를
# 수집한 데이터 입니다.
#수집된 피처 세트를 기반으로 결정 트리를 이용해 어떤 동작인지 예측해 보겠습니다.

In [2]:
# https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones
# train과 test 디렉터리에는 학습용 피처와 레이블 데이터 세트, 테스트용 피처와 클래스 값 데이터 세트가 들어있습니다.
# 피처는 561개가 있으며 공백으로 분리되어 있습니다.

In [3]:
# features.txt 파일은 피처 인덱스와 피처 명을 가지고 있으므로, DataFrame으로 로딩해 피처의 명칭만 확인해 보겠습니다.
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있음. 이를 dataframe으로 로드.
feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',header=None,names=['column_index','column_name'])
# read_csv() 는 pandas 에 속한 메소드이며 csv 파일을 읽어온다.
feature_name_df.info()
# 피처명 index를 제거하고, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   column_index  561 non-null    int64 
 1   column_name   561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [4]:
# 피처명을 보면 속성의 평균/표준편차가 X,Y,Z 축 값으로 돼 있음을 유추할 수 있습니다. 피처명을 가지는
# DataFrame을 이용해 데이터 파일을 로딩하기 전에 중복된 피처명을 처리해야 합니다.
# 따라서 중복된 피처명에 대해 원본 피처명에 _! 또는 _2를 추가하여 변경한 뒤 데이터를 DataFrame에 로드하겠습니다.

In [5]:
# 중복된 피처명이 얼마나 있는지 알아보겠습니다.
feature_dup_df = feature_name_df.groupby('column_name').count()
# 칼럼 이름 별로 grouping해서 count 하겠단 말씀

print(feature_dup_df[feature_dup_df['column_index']>1].count())

feature_dup_df[feature_dup_df['column_index']>1].head()

column_index    42
dtype: int64


,column_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3


In [6]:
# 총 42개의 feature 명이 중복돼 있습니다. 중복된 피처명에 대해 _1 또는 _2를 추가로 부여해 
#새로운 피처명을 갖는 DataFrame으로 변환하는 get_new_feature_name_df()를 생성합니다.
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                 columns=['dup_cnt'])
    # cumcount() 는 중복된 데이터에 대해 번호를 매겨준다.
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df
# 이제 train 디렉터리에 있는 학습용 피처데이터세트와 레이블 데이터 세트, test 디렉터리에 있는 텍스트용 피처 데이터 파일과 레이블 데이터 파일을 각각 학습/테스트용 
# DataFrame에 로드하겠습니다.
# 각 데이터 파일이 공백으로 분리돼 있으므로 read_csv()의 sep 인자로 공백 문자를 입력합니다.
# 레이블의 칼럽은 'action' 으로 명명하겠습니다. 
#  DataFrame을 생성하는 로직을 함수로 생성하겠습니다.


In [7]:
import pandas as pd
def get_human_dataset():
    # 데이터 파일은 공백으로 분리돼 있으므로 read_csv 에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                                 header=None,names=['column_index','column_name'])
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    # 불러온 파일을 함수를 통해 중복명을 처리한 후 새로운 df에 담기
    
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    #학습 피처 데이터세트와 테스트 피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name으로 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+',names=feature_name)
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 칼럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 df 모두 반환
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = get_human_dataset()

In [8]:
# 로드도니 학습용 피처 데이터 세트를 간략히 살핀다.
print("## 학습 피처 데이터셋 info()")
print(X_train.info())

## 학습 피처 데이터셋 info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None


In [9]:
X_train.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,...,0.125293,-0.307009,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,...,0.250994,0.321011,0.307584,0.336787,0.448306,0.608303,0.477975,0.511807,0.297480,0.279122
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.999873,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-0.995357,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.262975,-0.024863,-0.120993,-0.992754,-0.978129,-0.980233,-0.993591,-0.978162,-0.980251,-0.936219,...,-0.023692,-0.542602,-0.845573,-0.121527,-0.289549,-0.482273,-0.376341,-0.812065,-0.017885,-0.143414
50%,0.277193,-0.017219,-0.108676,-0.946196,-0.851897,-0.859365,-0.950709,-0.857328,-0.857143,-0.881637,...,0.134000,-0.343685,-0.711692,0.009509,0.008943,0.008735,-0.000368,-0.709417,0.182071,0.003181
75%,0.288461,-0.010783,-0.097794,-0.242813,-0.034231,-0.262415,-0.292680,-0.066701,-0.265671,-0.017129,...,0.289096,-0.126979,-0.503878,0.150865,0.292861,0.506187,0.359368,-0.509079,0.248353,0.107659
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,1.000000,0.478157,1.000000


In [10]:
print(y_train['action'].value_counts())

6    1407
5    1374
4    1286
1    1226
2    1073
3     986
Name: action, dtype: int64


In [11]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   action  7352 non-null   int64
dtypes: int64(1)
memory usage: 57.6 KB


In [12]:
# 레이블 값은 1~6 까지 6개의 값이고 분포한 걸 보면 거의 1000대로 균등하다
# DecisionTreeClassifier를 통해 동작 예측 분류를 수행해 보겠습니다.
# 하이퍼 파라미터 값은 디폴트 값으로 설정해 수행해 보고, 이때의 하이퍼 파라미터 값을 모두 추출해 보겠습니다.

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 동일한 예측결과 도출을 위해 random_state 설정
dt_clf = DecisionTreeClassifier(random_state=11)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

# DecisionTreeClassifier 의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터', dt_clf.get_params())

결정 트리 예측 정확도:0.8612
DecisionTreeClassifier 기본 하이퍼 파라미터 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 11, 'splitter': 'best'}


In [13]:
# 약 86%의 정확도입니다.

# 이번에는 결정 트리의 트리 깊이가 예측 정확도에 주는 영향을 살펴보겠습니다. 결정 트리의 경우 분류를 위해 리프 노드가 될 수 있는
# 적합한 수준이 될 때까지 지속해서 트리의 분할을 수행하면서 깊이가 깊어진다고 말했습니다.
# 다음은 GridSearchCV 를 이용해 사이킷런 결정 트리의 깊이를 조절할 수 있는 하이퍼 파라미터인 max_depth 값을 변화시키면서 예측 성능을 
# 확인해 보겠습니다. max_depth를 6,8,10,12,16,20,24 로 계속 늘리면서 예측 성능을 측정합니다. 교차 검증은 5개 세트입니다.

In [ ]:
from sklearn.model_selection import GridSearchCV
params ={
    'max_depth' : [6,7,10,12,16,20,24]
}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy',cv=5,verbose=1)
grid_cv.fit(X_train,y_train)
print('GridSearchCV 최고 평균 정확도 수치: {0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터: ', grid_cv.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


In [ ]:
# 어느 정도 깊이의 tree가 최적인지 판단해 봅니다. GridSearchCV는 tree에서 사용되며 지금 MAX DEPTH가 16일 때 가장 정확하다고 한다.

In [ ]:
# 이번에는 별도의 테스트 데이터 세트에서 DT의 정확도를 측정해 보겠습니다.
# 테스트 데이터 셑트에서 MAX_DEPTH에 따른 값을 측정해 보겠습니다.
max_depths = [6,7,10,12,16,20,24]
for depth in max_depths:
    dt_clf = DecisionTreeClassifier(max_depth=depth,random_state=11)
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    accuracy = accuracy_score(y_test,pred)
    print('max_depth ={0} 정확도: {1:4f}'.format(depth,accuracy))